In [42]:
from dataset import RelationIdentificationAndClassificationDataModule
dm = RelationIdentificationAndClassificationDataModule()
dm.setup("train")
train_dl = dm.train_dataloader()
val_dl = dm.val_dataloader()
# fetch all the data
train_d = list(train_dl) 
val_d = list(val_dl)

In [45]:
# unbatch 
unbatched_train = {
    k: [] for k in train_d[0].keys()
}
unbatched_val = {
    k: [] for k in train_d[0].keys()
}
for batch in train_d:
    for k, v in batch.items():
        unbatched_train[k] += v

for batch in val_d:
    for k, v in batch.items():
        unbatched_val[k] += v

# numpify
# for ds in [unbatched_train, unbatched_val]:
#     for k, v in ds.items():
#         ds[k] = np.array(v)
#     pass
train_d = unbatched_train
val_d = unbatched_val

In [5]:
from embedder import ContextBertEmbedder
embedder = ContextBertEmbedder()
embedded1 = embedder(
    to_embed=train_d["sentence1"],
    context=train_d["context"],
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0it [00:00, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (529 > 512). Running this sequence through the model will result in indexing errors


mismatch: 
 init: it is unwise to smoke in a restaurant as well as other public places, and relevant state laws should be codified to strictly prohibit it 
 found: it is unwise to smoke in a restaurant as well as other
mismatch: 
 init: as parents are involved in the same environmen 
 found: as parents are involved in the same environment, they could
mismatch: 
 init: mobile phones have shortened the distance of communicatio 
 found: mobile phones have shortened the distance of communication. after having considered
mismatch: 
 init: when children take jobs, they tend to be more responsibl 
 found: when children take jobs, they tend to be more responsible because whether they
mismatch: 
 init: if they make their plan in the first place, and then stick to their plans, they could get their work done quickly beyond their imagination 
 found: if they make their plan in the first place, and then stick to their plans, they could get their work done quickly beyond
mismatch: 
 init: in this co

In [6]:
embedded2 = embedder(
    to_embed=train_d["sentence2"],
    context=train_d["context"],
)

0it [00:00, ?it/s]

In [29]:
import torch
import numpy as np
embedded_concat = torch.concat([embedded1, embedded2], dim=-1)

In [26]:
extra1 = torch.cat(train_d["extra_features1"]).reshape(len(embedded_concat), -1)
extra2 = torch.cat(train_d["extra_features2"]).reshape(len(embedded_concat), -1)
shared = torch.cat(train_d["shared_features"]).reshape(len(embedded_concat), -1)

In [30]:
train = np.array(torch.concat([extra1, extra2, shared, embedded_concat], dim=-1))
train.shape

(15767, 4260)

In [31]:
embedded1_val = embedder(
    to_embed=val_d["sentence1"],
    context=val_d["context"],
)
embedded2_val = embedder(
    to_embed=val_d["sentence2"],
    context=val_d["context"],
)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [34]:
extra1 = torch.cat(val_d["extra_features1"]).reshape(len(embedded2_val), -1)
extra2 = torch.cat(val_d["extra_features2"]).reshape(len(embedded2_val), -1)
shared = torch.cat(val_d["shared_features"]).reshape(len(embedded2_val), -1)
val = np.array(torch.concat([extra1, extra2, shared, embedded1_val, embedded2_val], dim=-1))

In [35]:
train.shape, val.shape

((15767, 4260), (3941, 4260))

In [58]:
y_train = train_d["labels"]
y_train = np.array(torch.stack(y_train))
y_val = np.array(torch.stack(val_d["labels"]))

In [61]:
np.save("./ds/y_train", y_train)
np.save("./ds/y_val", y_val)
np.save("./ds/train", train)
np.save("./ds/val", val)

19708

In [62]:
y_train = np.load("./ds/y_train.npy")
y_val = np.load("./ds/y_val.npy")
train = np.load("./ds/train.npy")
val = np.load("./ds/val.npy")

In [63]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC()),
])

In [65]:
pipeline.fit(X=train, y=y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('svm', SVC())])

In [71]:
predict = pipeline.predict(X=val)

In [72]:
np.save("./ds/pred", predict)

In [76]:
from sklearn.metrics import f1_score
f1_score(y_val, predict, average=None, labels=[0, 1, 2])

array([0.70145931, 0.65951595, 0.08866995])